# Face Recognition with Tensorflow

### Teodor Kanev and Georgi Dimitrov

### 1. Problem

Face recognition is a method of identifying or verifying the identity of an individual from a digital image.
Our task for this project is to identify the exact position of the person face.

Example

### 2. Data source and preparation

<img src="https://lh3.googleusercontent.com/proxy/IFXjMln-UJ5JieNBR_vyHhUbzsT4dWJqHeqhvTXq0u3duGSTTTs8ezMUcyvLA9kzl4ug6pnMBfY-zJTEPdhzLpEp4T5P7zFIxlDAnRs">

The dataset we use can be found using the official website: http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html.

CelebFaces Attributes Dataset (CelebA) is a large-scale face attributes dataset with <b>202,599</b> celebrity images. For every image, we have 4 values (X-coordinate of top left corner, Y-coordinate of top left corner, width and height of the box) describing a rectangle covering the face.

First of all, let's import some libraries and write help functions.

In [ ]:
import os
import matplotlib.pyplot as plt 
import matplotlib.patches as patches
import numpy as np
from skimage import color
from skimage import io
import pandas as pd
from PIL import Image
import collections

In [ ]:
def to_index(name):
    return int(name[:6])

In [ ]:
def to_name(index):
    name = ''
    for i in range(6 - len(str(index))):
        name += '0'
    name = name + str(index) + '.jpg'
    return name

The next function extracts all images located in given folder to <b>images</b> with size 64x64 and the original
size is stored in <b>sizes</b>. Every item in <b>images</b> is with shape (64,64,3) using that every image is
in rgb format.

In [1]:
def load_rgb_images_from_folder(folder):
    images = {}
    sizes = {}
    for filename in os.listdir(folder):
        img = Image.open(os.path.join(folder,filename))
        if img is not None: 
            sizes[to_index(filename)] = img.size  
            new_img = img.resize((64,64), Image.ANTIALIAS)
            images[to_index(filename)] = new_img 
    return images, sizes

As we resize every image, we need to resize the bounding boxes aswell.

In [ ]:
def change_bbox(name, old_w, old_h, df):
    df.at[to_index(name)-1, 'x_1'] *= (64/old_w)
    df.at[to_index(name)-1, 'y_1'] *= (64/old_h)
    df.at[to_index(name)-1, 'width'] *= (64/old_w)
    df.at[to_index(name)-1, 'height'] *= (64/old_h)

Saving all images in <b>train_imgs</b>

In [ ]:
folder_name = 'Data/Part 1'
train_imgs, old_sizes = load_rgb_images_from_folder(folder_name)

Order them by image name in ordered dictionary.

In [ ]:
od = collections.OrderedDict(sorted(train_imgs.items()))

Save bounding boxes(which are sorted by image name) in data frame.

In [ ]:
df = pd.read_csv('Data/list_bbox_celeba.csv')

Scale bounding boxes for every image.

In [ ]:
for i in od.keys():
    img_number = i
    change_bbox(to_name(img_number), old_sizes[img_number][0], old_sizes[img_number][1], df)

Convert images to numpy array

In [ ]:
tst = []
for i in range(1,40000):
    tst.append(np.array(od[i]))

Use only needed bounding boxes.

In [ ]:
labels = np.array(df.head(39999).drop(columns = ['image_id'], axis = 1))[:]

Shuffle images and bounding boxes.

In [ ]:
from sklearn.utils import shuffle
images_shuffled, labels_shuffled = shuffle(tst, labels)

Split the dataset into: <br>
    <b> - Train data</b> - 80%<br>
    <b> - Validation data</b> - 10%<br>
    <b> - Test data</b> - 10%

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    images_shuffled, labels_shuffled, test_size=0.2, random_state=7)

X_val, X_test, y_val, y_test = train_test_split(
    X_val, y_val, test_size = 0.5, random_state = 7)

Convert everything to numpyp array.

In [ ]:
X_train = np.array(X_train)
Y_train = np.array(y_train)
X_val = np.array(X_val) 
Y_val = np.array(y_val)
X_test = np.array(X_test)
Y_test = np.array(y_test)

Import tensorflow libraries.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

The model architecture.

In [ ]:
model = keras.Sequential()
model.add(keras.Input(shape=(64, 64, 3)))
model.add(layers.Conv2D(64, kernel_size=(9, 9), strides=(1, 1), padding="same", activation = "relu"))

model.add(layers.BatchNormalization())

model.add(layers.Conv2D(64, kernel_size=(2, 2), strides=(2, 2), padding="same", activation = "relu"))
model.add(layers.Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding="same", activation = "relu"))
model.add(layers.BatchNormalization())

model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="valid"))
model.add(layers.Flatten())
model.add(layers.Dense(256))
model.add(layers.PReLU())
model.add(layers.Dense(4))

model.summary()
model.compile('Nadam', loss=tf.keras.losses.Huber())

Train the data

In [ ]:
model.fit(X_train, Y_train, epochs=10, validation_data=(X_val, Y_val), verbose=1, batch_size=64)

Function for Intersection over Union metric to determine the model corectness.

In [ ]:
def IOU(bbox1, bbox2):
    x1, y1, w1, h1 = bbox1[0], bbox1[1], bbox1[2], bbox1[3]
    x2, y2, w2, h2 = bbox2[0], bbox2[1], bbox2[2], bbox2[3]

    w_I = min(x1 + w1, x2 + w2) - max(x1, x2)
    h_I = min(y1 + h1, y2 + h2) - max(y1, y2)
    if w_I <= 0 or h_I <= 0:  # no overlap
        return 0.
    I = w_I * h_I

    U = w1 * h1 + w2 * h2 - I

    return I / U

Function to calculate overall accuracy of the model.

In [ ]:
def calc_accuracy(y_pred, y_test):
    tmp = []
    for i in range(len(y_pred)):
        tmp.append(IOU(y_pred[i], y_test[i]))
    
    return np.average(tmp)

Make predictions.

In [ ]:
y_pred = model.predict(X_test)

Results.

In [ ]:
calc_accuracy(y_pred, Y_test)

Function to draw prediction bounding box on the image.

In [ ]:
def draw_bbox(img, bbox):

    fig,ax = plt.subplots(1)

    # Display the image
    ax.imshow(img)

    # Create a Rectangle patch
    rect = patches.Rectangle((bbox[0],bbox[1]),bbox[2],bbox[3],linewidth=1,edgecolor='r',facecolor='none')

    # Add the patch to the Axes
    ax.add_patch(rect)

    plt.show()

Some results.

In [ ]:
draw_bbox[X_test[7], y_pred[7]]

In [ ]:
draw_bbox[X_test[17], y_pred[17]]

In [ ]:
draw_bbox[X_test[27], y_pred[27]]

Let's see where we fail to predict.

In [ ]:
results = []
for i in range(1,500):
    results.append([IOU(Y_test[i], y_pred[i]),i])
results.sort()
results

In [ ]:
draw_bbox(X_test[481], y_pred[481])

In [ ]:
draw_bbox(X_test[5], y_pred[5])

In [ ]:
draw_bbox(X_test[106], y_pred[106])

In [ ]:
draw_bbox(X_test[19], y_pred[19])